In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.offline import plot
from plotly.graph_objs import Scatter



import plotly as py
import pandas as pd
from plotly import session, tools, utils
import uuid
import json
from plotly import tools
import plotly.graph_objs as go

init_notebook_mode() 

In [2]:
def rectangledict(x0,y0,x1,y1,rgbcolorlinestr='rgb(10,10,10)',width=1,rgbfillcolor='rgb(100,250,250)'):
    out = {
        'type' : 'rect',
        'xref' : 'x',
        'yref' : 'y',
        'x0'   : x0,
        'y0'   : y0,
        'x1'   : x1,
        'y1'   : y1,
        'line' : {
            'color' : rgbcolorlinestr,
            'width' : width,
        },
        'fillcolor' : rgbfillcolor,
        'opacity' :0.9
    }
    return out
# x1 = s position of element in madx = end of element
# x0 = x1 - length of the element
# diff = (max val - minval)/10.
# y0 = max val + diff
# y1 = y0 + diff

def shapeannotations(xin,yin,textin):
    out = go.Scatter(
        x = [xin],
        y = [yin],
        text = textin,
        mode = 'markers',
        opacity = 0,
        hoverinfo = 'text',
        showlegend=False
    )
    return out

def get_plotlyjs():
    path = os.path.join('plotly.min.js')
    plotlyjs = resource_string('plotly', path).decode('utf-8')
    return plotlyjs


def new_iplot(figure_or_data, show_link=True, link_text='Export to plot.ly',
          validate=True):

    figure = tools.return_figure_from_figure_or_data(figure_or_data, validate)

    width = figure.get('layout', {}).get('width', '100%')
    height = figure.get('layout', {}).get('height', 525)
    try:
        float(width)
    except (ValueError, TypeError):
        pass
    else:
        width = str(width) + 'px'

    try:
        float(width)
    except (ValueError, TypeError):
        pass
    else:
        width = str(width) + 'px'

    plotdivid = uuid.uuid4()
    jdata = json.dumps(figure.get('data', []), cls=utils.PlotlyJSONEncoder)
    jlayout = json.dumps(figure.get('layout', {}), cls=utils.PlotlyJSONEncoder)

    config = {}
    config['showLink'] = show_link
    config['linkText'] = link_text
    jconfig = json.dumps(config)

    plotly_platform_url = session.get_session_config().get('plotly_domain',
                                                           'https://plot.ly')
    if (plotly_platform_url != 'https://plot.ly' and
            link_text == 'Export to plot.ly'):

        link_domain = plotly_platform_url\
            .replace('https://', '')\
            .replace('http://', '')
        link_text = link_text.replace('plot.ly', link_domain)


    script = '\n'.join([
        'Plotly.plot("{id}", {data}, {layout}, {config}).then(function() {{',
        '    $(".{id}.loading").remove();',
        '}})'
    ]).format(id=plotdivid,
              data=jdata,
              layout=jlayout,
              config=jconfig)

    html="""<div class="{id} loading" style="color: rgb(50,50,50);">
                 </div>
                 <div id="{id}" style="height: {height}; width: {width};" 
                 class="plotly-graph-div">
                 </div>
                 <script type="text/javascript">
                 {script}
                 </script>
                 """.format(id=plotdivid, script=script,
                           height=height, width=width)

    return html

In [3]:
import LHCclass
import madxmodule
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import csv
import glob
import datetime
import collections
import time
import subprocess
import os
from scipy import optimize as opt
from scipy import constants as const
from StringIO import StringIO
from matplotlib import rc,rcParams
from matplotlib.patches import Rectangle
import itertools

# simdata
from pandas.tools.plotting import autocorrelation_plot
from pandas.tools.plotting import lag_plot
from pandas.tools.plotting import scatter_matrix

s1 = time.time()

# loading measured data to compare with madx
# ------------------------------------------
fill4707 = LHCclass.LHCfill(4707,'/afs/cern.ch/work/t/tomerten/HI2015')
fill4696 = LHCclass.LHCfill(4696,'/afs/cern.ch/work/t/tomerten/HI2015')


# setting bump amplitude
# ----------------------
bumpamplist  = [-0.0005,0.000,0.001,0.002,0.003,0.004,0.005] # be careful for B1 signs needs to be opposite ->Beam4
npart        = 50

basefilelist =[['lhcb4-twiss-bfpp-'+ i + '.tfs'] for i in ['m0p5','0p0','1p0','2p0','3p0','4p0','5p0']] 

errorstringlist = ['']
corrlist        = np.array(['MCBCH.7L5.B2', 'MCBCH.9L5.B2', 'MCBH.13L5.B2'])


# twiss file list
# ---------------
basetwissfilelist = [[madxmodule.Twiss('lhcb2',basefilelist[j][i],targetxc=bumpamplist[j],IPcycle='IP5',
                                       targetel="MQ.11L5.B2",
                                       correctorlist=corrlist,errorseq=errorstringlist[i],
                                       twisscols=madxmodule.MADtwissColumns["LHCTwiss"],
                                       beam=[madxmodule.MADX_Beam(1,seq='LHCB1',energy=82.*6370.),
                                             madxmodule.MADX_Beam(2,seq='LHCB2',energy=82.*6370.)]) 
                      for i in range(len(basefilelist[j]))] for j in range(len(basefilelist))]


# initial parameters for BFPP
# ---------------------------
initialdictlist = [[madxmodule.get_initial(fn,madxmodule.dpPb(0,-1),location='IP5') 
                    for fn in basetwissfilelist[i]] for i in range(len(basetwissfilelist))]  


flist = [fn[0] for fn in [basetwissfilelist[i] for i in range(len(basetwissfilelist))]]

basedflist = [pd.read_csv(fn,
                skiprows=range(47),
                         delim_whitespace=True,
                         names =madxmodule.MADtwissColumns["LHCTwiss"])
              for fn in flist]


s2 = time.time()

In [4]:
flist = [fn[0] for fn in [basetwissfilelist[i] for i in range(len(basetwissfilelist))]]

basedflist = [pd.read_csv(fn,
                skiprows=range(47),
                         delim_whitespace=True,
                         names =madxmodule.MADtwissColumns["LHCTwiss"])
              for fn in flist]

In [141]:
basedflist[0]['APER_1']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
13236    0
13237    0
13238    0
13239    0
13240    0
13241    0
13242    0
13243    0
13244    0
13245    0
13246    0
13247    0
13248    0
13249    0
13250    0
13251    0
13252    0
13253    0
13254    0
13255    0
13256    0
13257    0
13258    0
13259    0
13260    0
13261    0
13262    0
13263    0
13264    0
13265    0
Name: APER_1, dtype: int64

In [5]:
fill4707.summarydf

,Description,Value,StartTime(UTC_TIME),EndTime(UTC_TIME)
0,Fill Number,4707,2015-12-08 15:48:28.101,2015-12-08 20:19:42.575
1,Beam Mode,SETUP,2015-12-08 16:22:39.039,2015-12-08 17:41:57.701
2,Beam Mode,INJPROT,2015-12-08 17:41:57.702,2015-12-08 17:59:39.258
3,Beam Mode,INJPHYS,2015-12-08 17:59:39.259,2015-12-08 18:51:30.275
4,Beam Mode,PRERAMP,2015-12-08 18:51:30.276,2015-12-08 18:54:43.555
5,Beam Mode,RAMP,2015-12-08 18:54:43.556,2015-12-08 19:15:11.137
6,Beam Mode,FLATTOP,2015-12-08 19:15:11.138,2015-12-08 19:19:04.762
7,Beam Mode,SQUEEZE,2015-12-08 19:19:04.763,2015-12-08 19:24:07.668
8,Beam Mode,SQUEEZE,2015-12-08 19:24:07.669,2015-12-08 19:37:07.426
9,Beam Mode,ADJUST,2015-12-08 19:37:07.427,2015-12-08 20:11:52.394


In [65]:
fill4707.lumicmsdf.max()

t    1.449606e+12
L    2.555217e-03
dtype: float64

# Plotting setup

In [6]:
globals()['rcParams']['font.size']=16.0
globals()['rcParams']['xtick.major.size']=12.0
globals()['rcParams']['ytick.major.size']=12.0
globals()['rcParams']['axes.labelsize']=12.0
globals()['rcParams']['xtick.labelsize']=12.0
globals()['rcParams']['ytick.labelsize']=12.0

# Plotting

In [7]:
def readtfs(tfsfile):
    df = pd.read_csv(tfsfile,skiprows=range(45),nrows=2,delim_whitespace=True)
    df = df[df['NAME']!='%s']
    colsdf = list(df.columns[1:])
    df = pd.read_csv(tfsfile,skiprows=range(47),delim_whitespace=True,names=colsdf,index_col=False)
    return df

In [8]:
def plotlybpm(LHCfill,tfslhcb1,tfslhcb2,simfile,ipnr=5,nn=50,step=20):
    bpmltest,namesl,tfsbpml,bpmrtest,namesr,tfsbpmr = LHCfill.transformbpmdata(
                                                    tfslhcb1,
                                                    tfslhcb2,
                                                    ipnr=ipnr
                                                   )
    from IPython.html.widgets import FloatProgress
    from IPython.display import display
    f = FloatProgress(min=0,max=2*nn/float(step))
    display(f)

    error = False
    
    df1 = readtfs(tfslhcb1)
    sip1=df1[df1['NAME']=='IP'+str(ipnr)]['S'].values[0]
    df2 = readtfs(tfslhcb2)
    sip2=df2[df2['NAME']=='IP'+str(ipnr)]['S'].values[0]
   
#     df1 = df1[['S','L','NAME','X','POLARITY','KEYWORD']]
#     df1['graph']=np.nan
#     diff = df1['X'].abs().max()+(df1['X'].abs().max()-df1['X'].abs().min())/10.*1000000
#     maximum = df1['X'].abs().max()*1000000
#     df1['graph']= df1.apply(lambda row: 
#                        rectangledict(row['S']-row['L']-sip1,
#                                     maximum+diff/2,
#                                     row['S']-sip1,
#                                     maximum+1.5* diff)
#                        if (str(row['NAME'])[0:2]=='MB') else None,
#                         axis = 1
#                        )
#     df1['graph']= df1.apply(lambda row: 
#                        rectangledict(row['S']-row['L']-sip1,
#                                     maximum+ diff,
#                                     row['S']-sip1,
#                                     maximum+ 2 * diff,
#                                     rgbfillcolor='rgb(0,0,255)')
#                        if ((row['KEYWORD']=='QUADRUPOLE')& (row['POLARITY']>0)) else row['graph'],
#                         axis = 1
#                        )

#     df1['graph']= df1.apply(lambda row: 
#                        rectangledict(row['S']-row['L']-sip1,
#                                     maximum,
#                                     row['S']-sip1,
#                                     maximum+ diff,
#                                     rgbfillcolor='rgb(255,0,0)')
#                        if ((row['KEYWORD']=='QUADRUPOLE')&(row['POLARITY']<0)) else row['graph'],
#                         axis = 1
#                        )

#     df1['graph']= df1.apply(lambda row: 
#                        rectangledict(row['S']-row['L']-sip1,
#                                     maximum+ 3.*diff/4.,
#                                     row['S']-sip1,
#                                     maximum+ 5.*diff/4.,
#                                     rgbfillcolor='rgb(153,255,204)')
#                        if (str(row['NAME'])[0:2]=='LE') else row['graph'],
#                         axis = 1
#                        )



#     df1['anno']= df1.apply(lambda row: 
#                        shapeannotations(row['S']-row['L']/2.-sip1,
#                                     maximum+diff/2.,
#                                     row['NAME'])
#                        if ((row['KEYWORD']=='QUADRUPOLE')|(str(row['NAME'])[0:2]=='MB')|(str(row['NAME'])[0:2]=='LE')) else None,
#                         axis = 1
#                        )

#     shapesb1    = list(df1[df1['graph'].notnull()]['graph'])
#     annotraceb1 = list(df1[df1['anno'].notnull()]['anno'])
    

    
#     df2 = df2[['S','L','NAME','X','POLARITY','KEYWORD']]
#     df2['graph']=np.nan
#     diff2 = df2['X'].abs().max()+(df2['X'].abs().max()-df2['X'].abs().min())/10.*1000000
#     maximum2 = df2['X'].abs().max()*1000000
#     df2['graph']= df2.apply(lambda row: 
#                        rectangledict(row['S']-row['L']-sip2,
#                                     maximum+diff/2,
#                                     row['S']-sip2,
#                                     maximum+1.5* diff)
#                        if (str(row['NAME'])[0:2]=='MB') else None,
#                         axis = 1
#                        )
#     df2['graph']= df2.apply(lambda row: 
#                        rectangledict(row['S']-row['L']-sip1,
#                                     maximum+ diff,
#                                     row['S']-sip1,
#                                     maximum+ 2 * diff,
#                                     rgbfillcolor='rgb(0,0,255)')
#                        if ((row['KEYWORD']=='QUADRUPOLE')& (row['POLARITY']>0)) else row['graph'],
#                         axis = 1
#                        )

#     df2['graph']= df2.apply(lambda row: 
#                        rectangledict(row['S']-row['L']-sip2,
#                                     maximum,
#                                     row['S']-sip2,
#                                     maximum+ diff,
#                                     rgbfillcolor='rgb(255,0,0)')
#                        if ((row['KEYWORD']=='QUADRUPOLE')&(row['POLARITY']<0)) else row['graph'],
#                         axis = 1
#                        )

#     df2['graph']= df2.apply(lambda row: 
#                        rectangledict(row['S']-row['L']-sip2,
#                                     maximum+ 3.*diff/4.,
#                                     row['S']-sip1,
#                                     maximum+ 5.*diff/4.,
#                                     rgbfillcolor='rgb(153,255,204)')
#                        if (str(row['NAME'])[0:2]=='LE') else row['graph'],
#                         axis = 1
#                        )



#     df2['anno']= df2.apply(lambda row: 
#                        shapeannotations(row['S']-row['L']/2.-sip2,
#                                     maximum+diff/2.,
#                                     row['NAME'])
#                        if ((row['KEYWORD']=='QUADRUPOLE')|(str(row['NAME'])[0:2]=='MB')|(str(row['NAME'])[0:2]=='LE')) else None,
#                         axis = 1
#                        )

#     shapesb2    = list(df2[df2['graph'].notnull()]['graph'])
#     annotraceb2 = list(df2[df2['anno'].notnull()]['anno'])

#     shapes = shapesb1 + shapesb2
#     annotrace = annotraceb1 + annotraceb2
    
    data = []
#     fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('BPMs LEFT IP5', 'Plot 2'))

    s1   = (-1.)* np.array([float(tfsbpml[tfsbpml['NAME']==n]['S'].values[0]) for n in namesl])+sip2
    max1 = len(bpmltest)
    
    

    for i in range(0,len(bpmltest.tail(nn)),step):
        try:
            trace = {
                'x': s1,
                'y':  np.array([bpmltest[n][max1-nn+i] for n in namesl]),
                'mode' : 'lines+markers',
                'name' : 'LEFT t'+str(i),
                'text' : namesl
            }
            data.append(trace)
            f.value = i
        except:
            print 'nn out of range'
            error = True
            break

    max2 = len(bpmrtest)
    s2   = np.array([float(tfsbpmr[tfsbpmr['NAME']==n]['S'].values[0]) for n in namesr])-sip1
    
    for i in range(0,len(bpmrtest.tail(nn)),step):
        try:
            trace = {
                'x': s2,
                'y':  np.array([bpmrtest[n][max2-nn+i] for n in namesr]),
                'mode' : 'lines+markers',
                'name' : 'RIGHT t'+str(i),
                'text' : namesr
            }
            data.append(trace)
            f.value = f.value +1 
        except:
            print 'nn out of range'
            error = True
            break
            
# only for ip5
#     trace = {
#                 'x': -simfile.S,
#                 'y': simfile['X'].apply(lambda t: -t*1000000.),
#                 'mode' : 'lines+markers',
#                 'name' : 'MADX',
#                 'text' : simfile['NAME']
#             }
#     data.append(trace)
    
#     for n in list(annotrace):
#         data.append(n)
  
    layout = go.Layout(
                title = 'BPMs IP'+str(ipnr)+' Fill ' +str(LHCfill.fillnumber),
                titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                autosize = False,
                width = 1500,
                height= 500,
                xaxis=dict(
                    title = 's [m from IP'+str(ipnr)+']',
                    titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                    range=[-1600,1600],
                    linewidth=4,
                    gridwidth =2,
#                     gridcolor = 'rgb(140,140,140)',
                    autotick=False,
                    ticks='outside',
                    tick0 =0,
                    dtick =100,
                    ticklen = 10,
                    #tickwidth = 2,
                    tickcolor='#000',
                    tickangle = 0,
                    showticklabels=True,
                    tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                    zeroline = True,
                    zerolinewidth=2
                 
                    #showgrid=True
                ),
                 yaxis=dict(
                    title = 'x',
                    titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                    range=[-5000,5000],
                    linewidth=4,
                    gridwidth =2,
#                     gridcolor = 'rgb(140,140,140)',
                    autotick=False,
                    ticks='outside',
                    tick0 =0,
                    dtick =500,
                    ticklen = 10,
                    #tickwidth = 2,
                    tickcolor='#000',
                    tickangle = 0,
                    showticklabels=True,
                    tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                    zeroline = True,
                    zerolinewidth=2
                 
                    #showgrid=True
                )
#         ,
#                 shapes = list(shapes)
            )

    fig = go.Figure(data=data, layout=layout)
#     print fig
    iplot(fig)
    html=new_iplot(fig,show_link=False)
    return html

In [ ]:
html4696ip5 = plotlybpm(fill4696,
        '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs',
        '/afs/cern.ch/work/t/tomerten/HI2015/lhcb2-twiss-noerr.tfs',
        basedflist[4],
        ipnr=5,nn=50,step=5)

In [ ]:
html4707ip5 = plotlybpm(fill4707,
        '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs',
        '/afs/cern.ch/work/t/tomerten/HI2015/lhcb2-twiss-noerr.tfs',
        basedflist[4],
        ipnr=5,nn=1000,step=80)

In [ ]:
html4696ip1 = plotlybpm(fill4696,
        '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs',
        '/afs/cern.ch/work/t/tomerten/HI2015/lhcb2-twiss-noerr.tfs',
        basedflist[4],
        ipnr=1,nn=50,step=5)

In [ ]:
html4707ip1 = plotlybpm(fill4707,
        '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs',
        '/afs/cern.ch/work/t/tomerten/HI2015/lhcb2-twiss-noerr.tfs',
        basedflist[4],
        ipnr=1,nn=50,step=5)

In [ ]:
html4696ip2 = plotlybpm(fill4696,
        '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs',
        '/afs/cern.ch/work/t/tomerten/HI2015/lhcb2-twiss-noerr.tfs',
        basedflist[4],
        ipnr=2,nn=50,step=5)

In [ ]:
html4707ip2 = plotlybpm(fill4707,
        '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs',
        '/afs/cern.ch/work/t/tomerten/HI2015/lhcb2-twiss-noerr.tfs',
        basedflist[4],
        ipnr=2,nn=50,step=5)

In [ ]:
bpml,namesl,tfsl, bpmr, namesr,tfsr = fill4707.transformbpmdata( '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs',
        '/afs/cern.ch/work/t/tomerten/HI2015/lhcb2-twiss-noerr.tfs',ipnr=5)

# bpms left of ip 5
tracel10 = {
        'x': range(len(bpml['BPM.10L5.B2'])),
        'y': bpml['BPM.10L5.B2'],
        'mode' : 'scattergl',
        'name' : 'BPM.10L5.B2',
        'text' : None
        }

tracel11 = {
        'x': range(len(bpml['BPM.11L5.B2'])),
        'y': bpml['BPM.11L5.B2'],
        'mode' : 'scattergl',
        'name' : 'BPM.11L5.B2',
        'text' : None
        }

tracel12 = {
        'x': range(len(bpml['BPM.12L5.B2'])),
        'y': bpml['BPM.12L5.B2'],
        'mode' : 'scattergl',
        'name' : 'BPM.12L5.B2',
        'text' : None
        }

# bpms right of ip5
tracer10 = {
        'x': range(len(bpmr['BPM.10R5.B1'])),
        'y': bpmr['BPM.10R5.B1'],
        'mode' : 'scattergl',
        'name' : 'BPM.10R5.B1',
        'text' : None
        }

tracer11 = {
        'x': range(len(bpmr['BPM.11R5.B1'])),
        'y': bpmr['BPM.11R5.B1'],
        'mode' : 'scattergl',
        'name' : 'BPM.11R5.B1',
        'text' : None
        }

tracer12 = {
        'x': range(len(bpmr['BPM.12R5.B1'])),
        'y': bpmr['BPM.12R5.B1'],
        'mode' : 'scattergl',
        'name' : 'BPM.12R5.B1',
        'text' : None
        }

fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Left IP5', 'Right IP5'))

fig.append_trace(tracel10, 1, 1)
fig.append_trace(tracel11, 1, 1)
fig.append_trace(tracel12, 1, 1)
fig.append_trace(tracer10, 1, 2)
fig.append_trace(tracer11, 1, 2)
fig.append_trace(tracer12, 1, 2)


fig['layout']['xaxis1'].update(title='time (index)',
                                titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                               linewidth=4,
                               gridwidth=2,
                               ticks='outside',
                               tick0 = 0,
                               dtick =500,
                               tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                                zeroline = True,
                                zerolinewidth=2
                              )
fig['layout']['yaxis1'].update(title='x[um]',
                               titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                               linewidth=4,
                               gridwidth=2,
                               ticks='outside',
                               tick0 = 0,
                               dtick =500,
                               tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                                zeroline = True,
                                zerolinewidth=2,
                               autorange=False,
                               range=[-4000,1000])

fig['layout']['xaxis2'].update(title='time (index)',
                                titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                               linewidth=4,
                               gridwidth=2,
                               ticks='outside',
                               tick0 = 0,
                               dtick =500,
                               tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                                zeroline = True,
                                zerolinewidth=2
                              )
fig['layout']['yaxis2'].update(title='x[um]',
                               titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                               linewidth=4,
                               gridwidth=2,
                               ticks='outside',
                               tick0 = 0,
                               dtick =500,
                               tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                                zeroline = True,
                                zerolinewidth=2,
                               autorange=False,
                               range=[-4000,1000])

fig['layout'].update(title='Time Evolution BPM fill 4707',titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ))
iplot(fig)
html4707evolutionbpm=new_iplot(fig,show_link=False)

In [ ]:
bpml,namesl,tfsl, bpmr, namesr,tfsr = fill4707.transformbpmdata( '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs',
        '/afs/cern.ch/work/t/tomerten/HI2015/lhcb2-twiss-noerr.tfs',ipnr=5)

baseline10     = bpml['BPM.10L5.B2'][9950]
baseline11     = bpml['BPM.11L5.B2'][9950]
baseline12     = bpml['BPM.12L5.B2'][9950]
indexlist    = [9500,9650,9700,9775,9810,9860,9900,10000]
difflist10   = [bpml['BPM.10L5.B2'][index]-baseline10 for index in indexlist]
difflist11   = [bpml['BPM.11L5.B2'][index]-baseline11 for index in indexlist]
difflist12   = [bpml['BPM.12L5.B2'][index]-baseline12 for index in indexlist]

thindexlist  = [10000,9900,9810,9700,9500] 
thbaseline10 = basedflist[1][basedflist[1]['NAME']=='BPM.10L5.B2']['X'].values[0]*10e5*(-1)
thbaseline11 = basedflist[1][basedflist[1]['NAME']=='BPM.11L5.B2']['X'].values[0]*10e5*(-1)
thbaseline12 = basedflist[1][basedflist[1]['NAME']=='BPM.12L5.B2']['X'].values[0]*10e5*(-1)
thdifflist10 = [basedflist[i][basedflist[i]['NAME']=='BPM.10L5.B2']['X'].values[0]*10e5*(-1)- thbaseline10 
                for i in range(len(basedflist)-2)]
thdifflist11 = [basedflist[i][basedflist[i]['NAME']=='BPM.11L5.B2']['X'].values[0]*10e5*(-1)- thbaseline11
                for i in range(len(basedflist)-2)]
thdifflist12 = [basedflist[i][basedflist[i]['NAME']=='BPM.12L5.B2']['X'].values[0]*10e5*(-1)- thbaseline12 
                for i in range(len(basedflist)-2)]

tracediff10 = {
        'x': indexlist,
        'y': difflist10,
        'mode' : 'markers',
        'marker': dict(
                        symbol = 'circle-open'),
        'name' : 'BPM.10L5.B2 - MEAS',
        'text' : None
        }

tracediff11 = {
        'x': indexlist,
        'y': difflist11,
        'mode' : 'markers',
        'marker': dict(
                        symbol = 'circle-open'),
        'name' : 'BPM.11L5.B2 - MEAS',
        'text' : None
        }

tracediff12 = {
        'x': indexlist,
        'y': difflist12,
        'mode' : 'markers',
        'marker': dict(
                        symbol = 'circle-open'),
        'name' : 'BPM.12L5.B2 - MEAS',
        'text' : None
        }

tracediffth10 = {
        'x': thindexlist,
        'y': thdifflist10,
        'mode' : 'markers',
        'marker': dict(
                        symbol = 'x'),
        'name' : 'BPM.10L5.B2 - MADX',
        'text' : None
        }

tracediffth11 = {
        'x': thindexlist,
        'y': thdifflist11,
        'mode' : 'markers',
        'marker': dict(
                        symbol = 'x'),
        'name' : 'BPM.11L5.B2 - MADX',
        'text' : None
        }

tracediffth12 = {
        'x': thindexlist,
        'y': thdifflist12,
        'mode' : 'markers',
        'marker': dict(
                        symbol = 'x'),
        'name' : 'BPM.12L5.B2 - MADX',
        'text' : None
        }

layout = go.Layout(
                title = 'Comparison relative (wrt zero bump) change in X[um] of BPM data and MADX theoretical prediction (fill 4707 - left of IP5)',
                titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 14,
                                    color = 'black'
                                    ),
                autosize = False,
                width = 1000,
                height= 500,
                xaxis=dict(
                    title = 'time (index)',
                    titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                    autorange=True,
                    linewidth=4,
                    gridwidth =2,
#                     gridcolor = 'rgb(140,140,140)',
                    autotick=False,
                    ticks='outside',
                    tick0 =0,
                    dtick =50,
                    ticklen = 10,
                    #tickwidth = 2,
                    tickcolor='#000',
                    tickangle = 0,
                    showticklabels=True,
                    tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                    zeroline = True,
                    zerolinewidth=2
                 
                    #showgrid=True
                ),
                 yaxis=dict(
                    title = 'x_diff',
                    titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                    autorange=True,
                    linewidth=4,
                    gridwidth =2,
#                     gridcolor = 'rgb(140,140,140)',
                    autotick=False,
                    ticks='outside',
                    tick0 =0,
                    dtick =200,
                    ticklen = 10,
                    #tickwidth = 2,
                    tickcolor='#000',
                    tickangle = 0,
                    showticklabels=True,
                    tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                    zeroline = True,
                    zerolinewidth=2
                 
                    #showgrid=True
                )
            )

data = [tracediff10,tracediff11,tracediff12,tracediffth10,tracediffth11,tracediffth12]
fig=go.Figure(data=data,layout=layout)
iplot(fig)
html4707relative=new_iplot(fig,show_link=False)
# plt.title('Comparison relative (wrt zero bump) change in X[um] of BPM data and MADX theoretical prediction (fill 4707 - left of IP5)',fontsize=16)
# # plt.rd('text',usetex=False)
# plt.xlabel('index of data')
# plt.ylabel('x_diff')
# plt.savefig('Rel_change_th_vs_bpm.png',format='png')
# plt.show()

In [ ]:
ddict = {
        'plt4696ip5':html4696ip5,'plt4696ip1':html4696ip1,'plt4696ip2':html4696ip2,
        'plt4707ip5':html4707ip5,'plt4707ip1':html4707ip1,'plt4707ip2':html4707ip2,
        'plt4707bpmevolution':html4707evolutionbpm,'plt4707bpmrelativemadx':html4707relative
        }
html_string = '''
<html>
    <head>
        <script type="text/javascript" src="mathjax/MAthJax.js?config=TeX-AMS-MML_SVG"></script>
        <script src="plotly-latest.min.js"></script>
    </head>
    <body>
    <h1>Summary of BPM data</h1>
    <h2>Fill 4696</h2>
    <h3>Fill 4696 - IP 1</h3>
    {plt4696ip1}
    <h3>Fill 4696 - IP 2/h3>
    {plt4696ip2}
    <h3>Fill 4696 - IP 5</h3>
    Note that this plot also contains MADX theoretical main beam4
    {plt4696ip5}
    <h2>Fill 4707</h2>
    <h3>Fill 4707 - IP 1</h3>
    {plt4707ip1}
    <h3>Fill 4707 - IP 2>/h3>
    {plt4707ip2}
    <h3>Fill 4707 - IP 5</h3>
    Note that this plot also contains MADX theoretical main beam4
    {plt4707ip5}
    <h2>BPM 10,11 and 12 time evolution left of IP5 for fill 4707 </h2>
    {plt4707bpmevolution}
    <h2>Comparison between theoretical and measured difference in beam positions for BPM 10,11 and 12 (IP5) for fill 4707 </h2>
    As during the quench test the BFPP bumps were reduced in steps from 3 mm to -0.5 mm I could compare the relative
    change in position of the main beam at BPMs 10,11 and 12 left of IP5 from the BPM measurements with the theoretical predictions
    from MADX. The plot below shows a good agreement of the measurements and the predictions.
    {plt4707bpmrelativemadx}
</body>
</html>'''.format(**ddict)
f = open('report.html','w')
f.write(html_string)
f.close()

In [ ]:
layout = go.Layout(
    xaxis=dict(
        domain=[0, 0.7]
    ),
    xaxis2=dict(
        domain=[0.8, 1]
    ),
    yaxis2=dict(
        anchor='x2'
    )
)
# fig = go.Figure(data=data, layout=layout)
trace2 = {
  "x": [20, 30, 40],
  "y": [50, 60, 70],
  "xaxis": 'x2',
  "yaxis": 'y2',
  'type': 'scatter'
};

trace1 = go.Scatter(x=[1, 2, 3], y=[4, 5, 6])

trace2 = go.Scatter(x=[20, 30, 40], y=[50, 60, 70])

trace3 = go.Scatter(x=[300, 400, 500], y=[600, 700, 800])

trace4 = go.Scatter(x=[4000, 5000, 6000], y=[7000, 8000, 9000])

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Plot 1', 'Plot 2',
                                                          'Plot 3', 'Plot 4'))

fig.append_trace(trace1, 1, 1)

fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

# All of the axes properties here: https://plot.ly/python/reference/#XAxis
fig['layout']['xaxis1'].update(title='xaxis 1 title')
fig['layout']['xaxis2'].update(title='xaxis 2 title', range=[10, 50])
fig['layout']['xaxis3'].update(title='xaxis 3 title', showgrid=False)
fig['layout']['xaxis4'].update(title='xaxis 4 title', type='log')

# All of the axes properties here: https://plot.ly/python/reference/#YAxis
fig['layout']['yaxis1'].update(title='yaxis 1 title')
fig['layout']['yaxis2'].update(title='yaxis 2 title', range=[40, 80])
fig['layout']['yaxis3'].update(title='yaxis 3 title', showgrid=False)
fig['layout']['yaxis4'].update(title='yaxis 4 title')

fig['layout'].update(title='Customizing Subplot Axes')
iplot(fig)

In [ ]:
def rectangledict(x0,y0,x1,y1,rgbcolorlinestr='rgb(10,10,10)',width=1,rgbfillcolor='rgb(100,250,250)'):
    out = {
        'type' : 'rect',
        'xref' : 'x',
        'yref' : 'y',
        'x0'   : x0,
        'y0'   : y0,
        'x1'   : x1,
        'y1'   : y1,
        'line' : {
            'color' : rgbcolorlinestr,
            'width' : width,
        },
        'fillcolor' : rgbfillcolor,
        'opacity' :0.9
    }
    return out
# x1 = s position of element in madx = end of element
# x0 = x1 - length of the element
# diff = (max val - minval)/10.
# y0 = max val + diff
# y1 = y0 + diff

def shapeannotations(xin,yin,textin):
    out = go.Scatter(
        x = [xin],
        y = [yin],
        text = textin,
        mode = 'markers',
        opacity = 0,
        hoverinfo = 'text',
        showlegend=False
    )
    return out

In [ ]:
trace = go.Scatter(
    x = [1.5,3],
    y = [2.5,2.5],
    text = 'test')
data = [trace]
shapes = []
anno = []

rect1 = rectangledict(1.,1.,2,2)
rect2 = rectangledict(3.,1.,5,2)
anno1 = shapeannotations(1.5,1.5,'recta')

anno2 = shapeannotations(4,1.5,'rectb')
shapes.append(rect1)
shapes.append(rect2)
data.append(anno1)
data.append(anno2)
print data
layout = {
    'shapes' : shapes
}
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [ ]:
df1 = readtfs( '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs')
df1 = df1[['S','L','NAME','X','POLARITY','KEYWORD']]
# df1 = df1[:50]
df1['graph']=np.nan

diff = df1['X'].abs().max()+(df1['X'].abs().max()-df1['X'].abs().min())/10.*1000000
maximum = df1['X'].abs().max()*1000000

df1['graph']= df1.apply(lambda row: 
                       rectangledict(row['S']-row['L'],
                                    maximum+diff/2,
                                    row['S'],
                                    maximum+1.5* diff)
                       if (str(row['NAME'])[0:2]=='MB') else None,
                        axis = 1
                       )
df1['graph']= df1.apply(lambda row: 
                       rectangledict(row['S']-row['L'],
                                    maximum+ diff,
                                    row['S'],
                                    maximum+ 2 * diff,
                                    rgbfillcolor='rgb(0,0,255)')
                       if ((row['KEYWORD']=='QUADRUPOLE')& (row['POLARITY']>0)) else row['graph'],
                        axis = 1
                       )

df1['graph']= df1.apply(lambda row: 
                       rectangledict(row['S']-row['L'],
                                    maximum,
                                    row['S'],
                                    maximum+ diff,
                                    rgbfillcolor='rgb(255,0,0)')
                       if ((row['KEYWORD']=='QUADRUPOLE')&(row['POLARITY']<0)) else row['graph'],
                        axis = 1
                       )

df1['graph']= df1.apply(lambda row: 
                       rectangledict(row['S']-row['L'],
                                    maximum+ 3.*diff/4.,
                                    row['S'],
                                    maximum+ 5.*diff/4.,
                                    rgbfillcolor='rgb(153,255,204)')
                       if (str(row['NAME'])[0:2]=='LE') else row['graph'],
                        axis = 1
                       )



df1['anno']= df1.apply(lambda row: 
                       shapeannotations(row['S']-row['L']/2.,
                                    maximum+diff/2.,
                                    row['NAME'])
                       if ((row['KEYWORD']=='QUADRUPOLE')|(str(row['NAME'])[0:2]=='MB')|(str(row['NAME'])[0:2]=='LE')) else None,
                        axis = 1
                       )

shapes=df1[df1['graph'].notnull()]['graph']
annotrace = df1[df1['anno'].notnull()]['anno']
# list(shapes)
# list(annotrace)

In [ ]:
trace = go.Scatter(
    x = df1['S'],
    y = df1['X']*1000000)

data = [trace]
shapes = []
rect1 = rectangledict(1.,1.,2,2)
rect2 = rectangledict(3.,1.,5,2)
shapes.append(rect1)
shapes.append(rect2)
for n in list(annotrace):
    data.append(n)

shapes=df1[df1['graph'].notnull()]['graph']
layout = {
    'xaxis':dict(
                    title = 's [m from IP]',
                    titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                    range=[0,500]),
    'shapes' : list(shapes)
}
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [ ]:
df2 = readtfs( '/afs/cern.ch/work/t/tomerten/HI2015/LHCTwiss-LHCB1.tfs')
df2[df2['NAME'].str[:2]=='MB'].head()

In [ ]:
df2.columns

# Fluka impact points

In [ ]:
416.2
417.2
418.3
419.0
421.5

In [9]:
fileextlist = [['noerr' + i] for i in ['m0p5','0p0','1p0','2p0','3p0','4p0','5p0'] ]

transfermatrixfilelist = [
    [madxmodule.TransferMatrix('LHCB2','IP5','S.DS.L5.B2',initialdictlist[j][i],bumpamplist[j],"MQ.11L5.B2",
                               corrlist,IPcycle='IP5',errorseq=errorstringlist[i],fileext=fileextlist[j][i],
                              beam=[madxmodule.MADX_Beam(1,seq='LHCB1',energy=82.*6370.),
                                    madxmodule.MADX_Beam(2,seq='LHCB2',energy=82.*6370.)]
                             ) for i in range(len(basetwissfilelist[j]))] for j in range(len(basefilelist))]

# writing transfermatrixfilelist to file for importing and plotting
dftransfermatrixfilelist       = [pd.DataFrame(pd.Series(data=transfermatrixfilelist[j]),columns=['filename']) 
                                  for j in range(len(transfermatrixfilelist))]

for j in range(len(dftransfermatrixfilelist)):
    dftransfermatrixfilelist[j].to_csv('transfermatrixfilenamelist'+ str(j) +'.csv',index=False)

opt         = madxmodule.io.tfsDict(basetwissfilelist[0][0])
prevelinex  = opt[0]["name"].index("MB.B11L5.B2")-1
s0          = opt[0]['s'][prevelinex]
print s0

403.8427102


In [10]:
# generating the filenames for storing the impactdistributions
impactfilelist    = [['impactIP5left_bp22_' + ext for ext in fileextlist[j]] for j in range(len(fileextlist))]
impactfilelistext = [[fn+'.csv' for fn in impactfilelist[j]] for j in range(len(impactfilelist))]

dfimpactfilelist  = [pd.DataFrame(pd.Series(data=impactfilelistext[j]),columns=['filename']) 
                     for j in range(len(impactfilelist))]

npart=5000


for j in range(len(dfimpactfilelist)):
    dfimpactfilelist[j].to_csv('impactfilenamelist' + str(j) +'.csv',index=False)
    
# generating the impact distributions
for j in range(len(transfermatrixfilelist)):
    for i in range(len(transfermatrixfilelist[j])):
        madxmodule.impactcoordinates6D(madxmodule.TrackSigmaMatrix(transfermatrixfilelist[j][i],
                                                                   'MB.B11L5.B2',initialdictlist[j][i],npart)
                        ,0.02315,madxmodule.lhcradius,s0,madxmodule.dipolelength,
                                       madxmodule.dpPb(0,-1),madxmodule.ionmass,
                                       -madxmodule.electronmassgev,
                                       madxmodule.get_p(transfermatrixfilelist[j][i]),
                                       impactfilelist[j][i],beam4=True)

MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2


In [18]:
data =[pd.read_csv(dfimpactfilelist[i]['filename'].values[0]) for i in range(len(dfimpactfilelist))]


bfppfnlist = [fn for fn in [dftransfermatrixfilelist[i]['filename'].values[0] for i in range(len(basetwissfilelist))]]

BFPPdflist = [pd.read_csv(fn,skiprows=range(47),
                delim_whitespace=True,names =madxmodule.MADtwissColumns["RMatrixExtended"]) 
          for fn in bfppfnlist]

[data[i].mean() for i in range(len(data))]


[s[m]           414.179114
 x[mm]          -23.142460
 px[1e-3]        -0.450955
 y[mm]            0.008205
 py[1e-3]        -0.000071
 E[GeV]      522339.279365
 dtype: float64, s[m]           414.939469
 x[mm]          -23.142736
 px[1e-3]        -0.447540
 y[mm]           -0.004765
 py[1e-3]        -0.000049
 E[GeV]      522338.820361
 dtype: float64, s[m]           416.549803
 x[mm]          -23.142770
 px[1e-3]        -0.440519
 y[mm]           -0.005127
 py[1e-3]         0.000001
 E[GeV]      522338.888981
 dtype: float64, s[m]           418.217791
 x[mm]          -23.142399
 px[1e-3]        -0.432711
 y[mm]           -0.001331
 py[1e-3]         0.000133
 E[GeV]      522336.158753
 dtype: float64, s[m]           420.008344
 x[mm]          -23.142656
 px[1e-3]        -0.419918
 y[mm]            0.009920
 py[1e-3]        -0.000099
 E[GeV]      522337.878567
 dtype: float64, s[m]           421.918367
 x[mm]          -23.142424
 px[1e-3]        -0.406258
 y[mm]           -0.001588
 p

In [19]:
beta1p = np.arctan(np.tan(data[2].mean()['px[1e-3]']*0.001)*(416.54-416.2)/(415.-416.2))

beta2p = np.arctan((np.tan(data[3].mean()['px[1e-3]']*0.001)*(418.23-417.2) 
                    - np.tan(data[2].mean()['px[1e-3]']*0.001)*(416.54-416.2))/(416.2-417.2))

beta3p = np.arctan((np.tan(data[4].mean()['px[1e-3]']*0.001)*(420.-418.3) 
                    - np.tan(data[3].mean()['px[1e-3]']*0.001)*(418.23-417.2))/(417.2-418.3))

beta4p = np.arctan((np.tan(data[5].mean()['px[1e-3]']*0.001)*(421.92-419.0) 
                    - np.tan(data[4].mean()['px[1e-3]']*0.001)*(420.-418.3))/(418.3-420.))

beta5p = np.arctan((np.tan(data[6].mean()['px[1e-3]']*0.001)*(423.91-421.5) 
                    - np.tan(data[5].mean()['px[1e-3]']*0.001)*(421.92-419.0))/(420.-421.92))

print beta1p,beta2p,beta3p,beta4p,beta5p

0.000124813855124 0.000295916212149 0.000243788191304 0.000277889756468 -0.000125506602204


In [ ]:
# genertate data points
from matplotlib.ticker import MultipleLocator,FormatStrFormatter

itab  = [-415.,-416.54,-418.23,-420.0,-421.92,-423.91]
iptab = [-415.,-416.20,-417.20,-418.3,-419.00,-421.50]
difftab = np.array(itab)- np.array(iptab)
# print difftab
y = [-0.02315+(np.tan(data[i].mean()['px[1e-3]']*0.001)) *difftab[i] for i in [1,2,3,4,5]]
y.append(-0.02315)
y.append(-0.02315)
y[:0]=[-0.02315,-0.02315]
iptab.append(-422)
iptab.append(-430)
iptab[:0]=[400,415.]

df2 = BFPPdflist[0].set_index('S')[['X','Y']]
ax = df2.plot(figsize=(16,8),grid=True,xticks=range(0,540,20))
for df in BFPPdflist:
    df2 = df.set_index('S')[['X','Y']]
    df2.plot(figsize=(16,8),grid=True,xticks=range(0,540,20),ax=ax)
    plt.xlim(0,500)
    plt.xlabel('s[m] from IP5 (left side)')
    plt.ylabel('x,y [mm]')
    plt.title('BFPP beam left of IP5')
    plt.plot(iptab[1:],y,color='black',linewidth=3.5)
plt.xlim(412,430)
plt.ylim(-0.033,-0.015)
plt.plot(np.array(iptab[1:])*(-1.),y)
ax.lines[-1].set_linewidth(4)
plt.plot([400,430],[-0.02315,-0.02315],color='black',linewidth=2)
minor_ticks = np.arange(-0.033,-0.015,0.001)
ax.set_yticks(minor_ticks,minor=True)
ax.grid(which='minor',alpha=0.5)

minor_xticks = np.arange(412,430,1)
ax.set_xticks(minor_xticks,minor=True)
ax.xaxis.set_minor_formatter(FormatStrFormatter('%d'))
majorlocator = MultipleLocator(100)
ax.xaxis.set_major_locator(majorlocator)
plt.savefig('BFPP_IP5_left_pipedeformation.png',format='png')
plt.show()

In [158]:
itab  = [-415.,-416.54,-418.23,-420.0,-421.92,-423.91]
iptab = [-415.,-416.20,-417.20,-418.3,-419.00,-421.50]
difftab = np.array(itab)- np.array(iptab)
print difftab
y = [-0.02315+(np.tan(data[i].mean()['px[1e-3]']*0.001)) *difftab[i] for i in [1,2,3,4,5]]
y.append(-0.02315)
y.append(-0.02315)
y[:0]=[-0.02315,-0.02315]
iptab.append(-422)
iptab.append(-430)
iptab[:0]=[400,415.]

dataly = []
for df in BFPPdflist:
    trace = go.Scatter(
                x = -1. * df['S'],
                y = -1.0* df['X'] * 1000000.,
                mode = 'lines+markers',
                text = df['NAME'],
                name = 'BFPP'
            )
    dataly.append(trace)

df2 = readtfsaper( '/afs/cern.ch/work/t/tomerten/sixtrack/LHCB2n1.tfs')
df2 = df2[df2['APER_1'] >0]
sip = df2[df2['NAME']=='IP5']['S'].values[0]

trace = go.Scatter(
                x =  df2['S']-sip,
                y =  df2['APER_1'] * 1000000.,
                mode = 'lines',
                text = df2['NAME'],
                name = 'Aperture'
            )
dataly.append(trace)

trace = go.Scatter(
                x =  df2['S']-sip,
                y = [60000.]*len(df2),
                fill = 'tonexty',
                mode = 'lines',
                name = 'Aperture',
                hoverinfo = False
            )
dataly.append(trace)

layout = dict(
                title = 'BFPP left IP5 for different bumps',
                titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                autosize = False,
                width = 1500,
                height= 500,
                xaxis=dict(
                    title = 's [m from IP',
                    titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                    range=[-500,0],
                    linewidth=4,
                    gridwidth =2,
#                     gridcolor = 'rgb(140,140,140)',
                    autotick=False,
                    ticks='outside',
                    tick0 =0,
                    dtick =100,
                    ticklen = 10,
                    #tickwidth = 2,
                    tickcolor='#000',
                    tickangle = 0,
                    showticklabels=True,
                    tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                    zeroline = True,
                    zerolinewidth=2
                 
                    #showgrid=True
                ),
                 yaxis=dict(
                    title = 'x',
                    titlefont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 16,
                                    color = 'black'
                                    ),
                    range=[-5000,50000],
                    linewidth=4,
                    gridwidth =2,
#                     gridcolor = 'rgb(140,140,140)',
                    autotick=False,
                    ticks='outside',
                    tick0 =0,
                    dtick =5000,
                    ticklen = 10,
                    #tickwidth = 2,
                    tickcolor='#000',
                    tickangle = 0,
                    showticklabels=True,
                    tickfont = dict(
                                    family = 'Arial, sans-serif',
                                    size = 12,
                                    color = 'black'
                                    ),
                    zeroline = True,
                    zerolinewidth=2
                 
                    #showgrid=True
                )
        ,
                shapes = [{
                        'type' : 'line',
                        'x0': 0,
                        'y0': 23150,
                        'x1':500,
                        'y1': 23150, 
                        'line' : {'width' : 4}}]
            )
fig = go.Figure(data=dataly, layout=layout)
#     print fig
iplot(fig)
# html=new_iplot(fig,show_link=False)


[ 0.   -0.34 -1.03 -1.7  -2.92 -2.41]


In [135]:
df2 = readtfs('/afs/cern.ch/work/t/tomerten/sixtrack/lhcb4-twiss-bfpp-0p0.tfs')

df2.columns

Index([u'NAME', u'KEYWORD', u'PARENT', u'S', u'L', u'LRAD', u'KICK', u'HKICK',
       u'VKICK', u'ANGLE', u'K0L', u'K1L', u'K2L', u'K3L', u'X', u'Y', u'PX',
       u'PY', u'BETX', u'BETY', u'ALFX', u'ALFY', u'MUX', u'MUY', u'DX', u'DY',
       u'DPX', u'DPY', u'KMAX', u'KMIN', u'CALIB', u'POLARITY', u'APERTYPE',
       u'APER_1', u'APER_2', u'APER_3', u'APER_4', u'N1', u'TILT'],
      dtype='object')

In [ ]:
aperturefileb2 = '/afs/cern.ch/eng/lhc/optics/V6.503/aperture/lhc_mech_axis_fidshifts-b2.madx'


In [130]:
apertureelementlist = pd.read_csv('/afs/cern.ch/user/t/tomerten/aperture/namefile.csv',header=None,names=['NAME'])
apertureelementlist = list(apertureelementlist['NAME'])
print len(apertureelementlist)

6147


In [134]:
list(set(apertureelementlist)-set(list(df2['NAME'])))

['BPMS.2R5.B1',
 'BPMS.2L2.B1',
 'BPMC.8R4.B2',
 'MQT.27R3.B2',
 'BPMCA.7R4.B2',
 'MQT.23R3.B2',
 'BPMC.9R4.B2',
 'BPMS.2L8.B1',
 'BPMC.9L4.B2',
 'BPMC.10L4.B2',
 'BPMC.10R4.B2',
 'BPMS.2R8.B1',
 'BPMS.2L5.B1',
 'BPMS.2R2.B1',
 'BPMC.8L4.B2',
 'BPMC.7L4.B2',
 'BPMS.2R1.B1',
 'BPMS.2L1.B1']

In [131]:
df2[df2['NAME'].isin(apertureelementlist)]

6129

In [144]:
def readtfsaper(tfsfile):
    df = pd.read_csv(tfsfile,skiprows=range(30),nrows=2,delim_whitespace=True)
    df = df[df['NAME']!='%s']
    colsdf = list(df.columns[1:])
    df = pd.read_csv(tfsfile,skiprows=range(32),delim_whitespace=True,names=colsdf,index_col=False)
    return df

aperdfs = readtfsaper('/afs/cern.ch/work/t/tomerten/sixtrack/LHCB2n1.tfs')

In [154]:
aperdfs[aperdfs['NAME']=='IP5']

,NAME,S,APER_1,APER_2,APER_3,APER_4,BETX,BETY,DX,DY,X,Y,RTOL,XTOL,YTOL
21845,IP5,13329.59397,0.029,0.029,0.029,0.029,11.0,11.000001,0.0179,0.026475,5.361109e-10,-0.002,0.011,0.0,0.0
